In [69]:
import pickle
import requests
from bs4 import BeautifulSoup
import re
import os
import os.path
import base64
import urllib.parse
import time

with open('mfct_urls.pickle', 'rb') as f:
    mfct_urls = pickle.load(f)
with open('expanded_urls.pickle', 'rb') as f:
    expanded_urls = pickle.load(f)
with open('all_models.pickle', 'rb') as f:
    all_models = pickle.load(f)

In [70]:
import collections
assert(len([item for item, count in collections.Counter(all_models).items() if count > 1]) == 0)


In [71]:
cache_dir = os.path.expanduser("cache/")

def retrieve_and_cache(url,verbose=False,session=None):
    # pass in a requests.Session() object if you want to make many calls, keeping the connection open.
    # This gives you a 2x speedup.
    
    # This caching has a MAJOR FLAW: It does not check whether the page has been updated.
    # Once downloaded, it will keep the file forever. 
    # This is acceptable since this is to aid development when I don't want to repeatedly hit the 
    # server and wait for a download, but, this is NOT well-suited for scraping the site to check for updates.
    # I suggest deleting the cache folder before doing a big scrape
    #
    # We can also try out CacheControl https://cachecontrol.readthedocs.io/en/latest/
    
    start_time = time.time()
    if not os.path.exists(cache_dir):
        if verbose:
            print("Initializing cache directory")
        os.makedirs(cache_dir)
    filename = urllib.parse.quote(url, '')
    filepath = os.path.join(cache_dir, filename)
    if verbose:
        print("Checking cache for", filename)
    raw_html = ""

    if os.path.exists(filepath):
        if verbose:
            print("  Retrieving from cache")
        with open(filepath, 'r') as file:
            raw_html = file.read()
    else:
        if verbose:
            print("  Fetching and saving to cache")
        response = None
        if session:
            response = session.get(url)
        else:
            response = requests.get(url)
        
        raw_html = response.text
        with open(filepath, 'x') as file:
            file.write(raw_html)
    if verbose:
        print("  Took", time.time() - start_time,"seconds")
    return raw_html

def dedupe_list_preserving_order(items):
    return list(dict.fromkeys(items))

def parse_table_to_dict(text,url=None): 
    # Parses the first Make-Model table we find on the page.
    # Returns an empty {} if we didn't find any data.
    soup = BeautifulSoup(text, 'lxml') # important to pip install lxml
    output = {}
    
    # Find the Table
    hook = soup.find_all(string=re.compile(".*Make Model.*"))
    if len(hook) == 0:
        return output
    if len(hook) > 1:
        print("Woah more than one \"Make Model\" entry, using the first one inside a table [",url,"]")
    assert(len(hook)>=1)  # Assert we find at least one NavigatableString containing "Make Model"
    table = None
    for h in hook: # Let's find the first one that is inside a table. 
        if h.find_parent('table'):
            table = h.find_parent('table')
            break
    assert(table) # We assert we did find one
    rows = table.find_all('tr')
    assert(len(rows) > 1) # Assert we've found and parsed a table with many rows
    
    # Extract the key:value pairs
    for row in rows:
        cols = row.find_all('td')
        if len(cols) != 2:
            print("We did not find 2 columns in this row, skipping... [", url, "]")
            continue
        assert(len(cols) == 2) # We expect two columns in each row, which we treat as a key:value pair
        key = cols[0].get_text().strip()
        value = cols[1].get_text().strip().replace("\t", "")
        output[key] = value
    return output

def decorate_table(table_dict, key_vals):
    return table_dict | key_vals

def get_mfct(name):
    for i, m in enumerate(mfct_urls):
        if m['mfct'] == name:
            return m
    return None
        
def clean_keys(table_dict,url=None):
    d = {}
    for k, v in table_dict.items():
        # Remove punctuation 
        k = k.translate(str.maketrans('', '', '!"#$%&\'()*+,-.:;<=>?@[\]^_`{|}~'))
        
        # Remove extraneous whitespace
        k = " ".join(k.split()).strip()
        
        # Ensure Title Case
        k = k.title()

        # Collapse certain special cases. DANGER, THIS WILL COLLIDE TABLE ENTRIES
        k = k.replace("Abs", "ABS")
        k = k.replace("Rear Wheels", "Rear Wheel")
        k = k.replace("Wheels Rear", "Rear Wheel")
        k = k.replace("Front Wheels", "Front Wheel")
        k = k.replace("Wheels Front", "Front Wheel")
        k = k.replace("Final Reduction Ratio", "Final Reduction")
        k = k.replace("Gear Ratios", "Gear Ratio")
        k = k.replace("Gear Tatio", "Gear Ratio")
        k = k.replace("Primary Reduction Ratio", "Primary Reduction")
        k = k.replace("Dryweight", "Dry Weight")
        k = k.replace("Dry Weight /", "Dry Weight")
        k = k.replace("Wetweight", "Wet Weight")
        k = k.replace("Wet Wight", "Wet Weight")
        k = k.replace("We Weight", "Wet Weight")
        k = k.replace("Final Srive", "Final Drive")
        k = k.replace("Final Drice", "Final Drive")
        k = k.replace("Final Ldrive", "Final Drive")
        #k = k.replace("Final Driv", "Final Drive")  # I NEED REGEX HERE ELSE WE PLURIFY EVERYTHING
        #k = k.replace("Dimension", "Dimensions")    # I NEED REGEX HERE ELSE WE PLURIFY EVERYTHING
        k = k.replace("Seat Hieght", "Seat Height")
        k = k.replace("Seat Heigth", "Seat Height")
        k = k.replace("Gouun Clearance", "Ground Clearance")
        k = k.replace("Staring", "Starting")
        k = k.replace("Rear Wheel Travcel", "Rear Wheel Travel")
        k = k.replace("Front Wheel Travlel", "Front Wheel Travel")
        k = k.replace("Lubricationn", "Lubrication")
        
        
        
        # Save the new key
        if k in d:
            # Combine entries if there's a colliding key. If no collision, we ignore.
            if d[k].strip() != v.strip():
                #print("Found colliding entries for key", k, "combining old value [",d[k],"] and new value [",v,"] for",url)
                d[k] = d[k] + " | " + v
        else:
            d[k] = v
                
        
    return d

def clean_vals(table_dict,url=None):
    d = {}
    for k, v in table_dict.items():
        # Remove extraneous whitespace
        v = " ".join(v.split()).strip()

        # Save the new key
        d[k] = v
        
    return d

def clean_model_table(table_dict,url=None):
    return clean_vals(clean_keys(table_dict,url=url),url=url)

In [72]:
def get_table_for_url(url, return_empty_tables=True):
    raw_html = retrieve_and_cache(url)
    table_dict = parse_table_to_dict(raw_html,url=url)
    if table_dict == {} and not return_empty_tables:
        return {}
    table_dict_decorated = decorate_table(table_dict, {"Link":url})
    table_dict_decorated_cleaned = clean_model_table(table_dict_decorated,url=url)
    return table_dict_decorated_cleaned

In [73]:
def get_all_model_tables_for_mfct(mfct_name):
    m = get_mfct(mfct_name)
    tables = []
    for url in m['models']:
        table = get_table_for_url(url, return_empty_tables=False)
        if table != {}:
            tables.append(table)
    return tables

In [74]:
cleaned_tables = get_all_model_tables_for_mfct('Kawasaki')

We did not find 2 columns in this row, skipping... [ https://www.motorcyclespecs.co.za/model/kawasaki/kawasaki_s3_400%2075.htm ]
We did not find 2 columns in this row, skipping... [ https://www.motorcyclespecs.co.za/model/kawasaki/kawasaki_s3_400%2075.htm ]


In [75]:
import pandas as pd
df = pd.DataFrame(cleaned_tables)
df

,Make Model,Year,Engine,Capacity,Bore X Stroke,Cooling System,Compression Ratio,Induction,Ignition,Starting,...,Reviews,Standing 0 100Km,Standing 0 140Km,Standing 0 200Km,Acceleration 60140 Km/H,Acceleration 100 140 Km/H,Acceleration 140 180 Km/H,Firing Order,Minimum Turning Circle,Related Links
0,Kawasaki 250 A1 Samurai,1966 - 68,"Two-stroke, parallel twin, 2 rotary inlet valves",247 cc / 15.1 cu-in,53 x 56.0 mm,Air cooled,7.0:1,Carburetor,Capacitor discharge ignition,Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kawasaki 250 A1 Samurai,1969 -71,"Two-stroke, parallel twin, 2 rotary inlet valves",247 cc / 15.1 cu-in,53 x 56.0 mm,Air cooled,7.0:1,Carburetor,Capacitor discharge ignition,Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kawasaki AR 50,1981 - 96,"Two stroke, single cylinder, disc valve.",49 cc / 3.0 cu-in,49 x 41.6 mm,Air cooled,7.8:1,Mikuni VM18SC Carburetor,Capacitor discharge,Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kawasaki AR 80,1981 -83,"Two stroke, single cylinder, disc valve.",79.4 cc / 4.8 cu-in,49 x 41.6 mm,Air cooled,7.8:1,Mikuni VM18SC Carburetor,Capacitor discharge,Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Kawasaki AR 80,1984 -,"Two stroke, single cylinder, disc valve.",79.4 cc / 4.8 cu-in,49 x 41.6 mm,Air cooled,7.8:1,Mikuni VM18SC Carburetor,Capacitor discharge,Kick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,Kawasaki ZZ-R 1200 / ZX-12C,2004,"Four stroke, transverse four cylinder, DOHC, 4...",1164 cc / 71.0 cu-in,78 x 59 mm,Liquid cooled,10.6:1,"4x Keihin CVKD40, TPS",Digital ignition,Electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,Kawasaki ZZ-R 1200 / ZX-12C,2005,"Four stroke, transverse four cylinder, DOHC, 4...",1164 cc / 71.0 cu-in,78 x 59 mm,Liquid cooled,10.6:1,"4x Keihin CVKD40, TPS",Digital ignition,Electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,Kawasaki ZZR 1400 (ZX-14) Special Edition,2007,"Four stroke, transverse four cylinders, DOHC, ...",1441 cc / 87.9 cu-in,84 x 65 mm,Liquid cooled,12.3:1,DFI® with four 44mm Mikuni throttle bodies,TCBI with Digital Advance,Electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,Kawasaki ZZR 1400 Performance Sport,2014,"Four stroke, transverse four cylinders, DOHC, ...",1441 cc / 87.9 cu-in cc / 87.9 cu-in,84 x 65 mm,Liquid cooled,12.3:1,DFI® with four 44mm Mikuni throttle bodies,TCBI with Digital Advance,Electric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df.to_csv('kawi-no-empty.csv')

## WARNING: THE CODE BELOW PROCESSES ALL 11k MODELS AND SAVES TO OVER 100 CSV FILES

In [36]:
all_tables = []
for i, mfct_name in enumerate([m['mfct'] for m in mfct_urls]):
    print("Processing", i,"of",len(mfct_urls)-1,":",mfct_name)
    cleaned_tables = get_all_model_tables_for_mfct(mfct_name)
    all_tables.extend(cleaned_tables)
    df = pd.DataFrame(cleaned_tables)
    mfct_urls[i]['df'] = df
    df.to_csv("models {index:03d} - {mfct_name}.csv".format(index = i, mfct_name=urllib.parse.quote(mfct_name,'')))

Processing 0 of 131 : AC Schnitzer
Processing 1 of 131 : Adler
Processing 2 of 131 : AJP
Processing 3 of 131 : AJS
Processing 4 of 131 : Alfer
Processing 5 of 131 : Aprilia
Found colliding entries for key Final Drive combining old value [ Chain ] and new value [ 17:40 ] for https://www.motorcyclespecs.co.za/model/aprilia/aprilia-rs125r-nero-06.html
Found colliding entries for key Final Drive combining old value [ Chain ] and new value [ 17:40 ] for https://www.motorcyclespecs.co.za/model/aprilia/aprilia-rs125r-06.html
Found colliding entries for key Final Drive combining old value [ Chain ] and new value [ 17:40 ] for https://www.motorcyclespecs.co.za/model/aprilia/aprilia-rs125r-07.html
Found colliding entries for key Final Drive combining old value [ Chain ] and new value [ 17:40 ] for https://www.motorcyclespecs.co.za/model/aprilia/aprilia-rs125-lorenzo-07.html
Found colliding entries for key Final Drive combining old value [ Chain ] and new value [ 17:40 ] for https://www.motorcycl

We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/ducati/ducati_multistrada_1200%20S%2013.htm
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/ducati/Ducati_Multistrada_1200S_Touring_13.html
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/ducati/ducati_multistrada_1200%20S%20GT%2013.htm
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/ducati/ducati_multistrada_1200%20S%20pikes%20peak%2013.htm
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/ducati/ducati_multistrada_1200%20SDolomites%20Peak%20Edition.htm
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/ducati/ducati_multistrada_1200%20S%2014.htm
We found more than 2 columns, using only the first two, on page htt

Processing 56 of 131 : Horex
Processing 57 of 131 : HPN
Processing 58 of 131 : Husaberg
Processing 59 of 131 : Husqvarna
Processing 60 of 131 : Hyosung
Processing 61 of 131 : Indian
Processing 62 of 131 : Italjet
Processing 63 of 131 : Jawa
Processing 64 of 131 : Junak
Processing 65 of 131 : Kawasaki
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/kawasaki/kawasaki_s3_400%2075.htm
We found more than 2 columns, using only the first two, on page https://www.motorcyclespecs.co.za/model/kawasaki/kawasaki_s3_400%2075.htm
Found colliding entries for key Ground Clearance combining old value [ 1435 mm / 56.5 in. ] and new value [ 125 mm / 4.9 in ] for https://www.motorcyclespecs.co.za/model/kawasaki/kawasaki_z1000r_17.htm
Found colliding entries for key Ground Clearance combining old value [ 1435 mm / 56.5 in. ] and new value [ 125 mm / 4.9 in ] for https://www.motorcyclespecs.co.za/model/kawasaki/kawasaki_z1000r_19.html
Processing 66 of 

# Combine all the tables into one giant table

In [10]:
df = pd.DataFrame(all_tables)

# Export to CSV

In [13]:
df.to_csv('ALL_MODELS.csv')

# Export to Excel

Install the dependencies

> pip install XlsxWriter


In [24]:
#with pd.ExcelWriter("ALL_MODELS.xlsx") as writer:
#    df.to_excel(writer, sheet_name="models", index=False)

with pd.ExcelWriter("models_on_sheets.xlsx") as writer:
    for m in mfct_urls:
        m['df'].to_excel(writer, sheet_name=urllib.parse.quote(m['mfct'],'')[0:31], index=False)

# Debugging

In [133]:
url = "https://www.motorcyclespecs.co.za/model/voxan/Voxan_GTV_1200.html"
raw_html = retrieve_and_cache(url)
table_dict = parse_table_to_dict(raw_html,url=url)


Woah more than one table at https://www.motorcyclespecs.co.za/model/voxan/Voxan_GTV_1200.html


AttributeError: 'NoneType' object has no attribute 'find_all'

In [142]:
soup = BeautifulSoup(raw_html, 'lxml') # important to pip install lxml
output = {}

# Find the Table
hook = soup.find_all(string=re.compile(".*Make Model.*"))
if len(hook) > 1:
    print("Woah more than one table at",url)
assert(len(hook)>=1)  # Assert we find at least one NavigatableString containing "Make Model"
for h in hook:
    if h.find_parent('table'):
        table = h.find_parent('table')
        break
table


Woah more than one table at https://www.motorcyclespecs.co.za/model/voxan/Voxan_GTV_1200.html


<table height="821" id="table8" width="100%">
<tr>
<td class="xl24" height="39" style="border-style: solid" width="618">
<p align="justify"><font face="Arial">Make Model</font></p></td>
<td class="xl25" height="38" style="border-left-style: solid; border-left-width: medium; border-right-style: solid; border-top-style: solid; border-bottom-style: solid" width="1230">
<font face="Arial">Voxan GTV 1200</font></td>
</tr>
<tr>
<td class="xl24" height="39" style="border-left-style: solid; border-right-style: solid; border-top-style: solid; border-top-width: medium; border-bottom-style: solid" width="618">
<p align="justify"><font face="Arial">Year</font></p></td>
<td class="xl26" height="38" style="border-left-style: solid; border-left-width: medium; border-right-style: solid; border-top-style: solid; border-top-width: medium; border-bottom-style: solid" width="1230" x:num="">
<font face="Arial">2010</font></td>
</tr>
<tr>
<td class="xl24" height="39" style="border-left-style: solid; border-

In [141]:
hook

['Make Model', 'Make Model']

In [ ]:
hook = hook[0] # For now we're only parsing the first one. 
table = hook.find_parent('table')
rows = table.find_all('tr')


In [1]:
import xlsxwriter

In [23]:
"hkjansdflkjnasdf"[0:31]

'hkjansdflkjnasdf'